In [5]:
import pandas as pd
import torch
from transformers import T5ForConditionalGeneration, T5TokenizerFast
from google.colab import drive
import os
import random
from functools import lru_cache # Import lru_cache

# ==============================================================================
# 0) CONFIGURATION
# ==============================================================================
# Define paths for your data and models in Google Drive
CLUSTERED_DATA_PATH = '/content/drive/MyDrive/amazon_reviews_clustered_and_categorized.csv'
SUMMARIZATION_MODEL_NAME = "t5-small" # You can use "t5-base" for better quality but slower generation
OUTPUT_BLOG_POSTS_DIR = '/content/drive/MyDrive/generated_blog_posts/'

# Set device for model inference
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {DEVICE}")

# ==============================================================================
# 1) MOUNT GOOGLE DRIVE
# ==============================================================================
print("Mounting Google Drive...")
try:
    drive.mount('/content/drive')
    print("Google Drive mounted successfully.")
except Exception as e:
    print(f"Error mounting Google Drive: {e}")
    print("Please ensure you are running in a Colab environment and authorize access.")

# ==============================================================================
# 2) LOAD DATA AND SUMMARIZATION MODEL
# ==============================================================================
@lru_cache(maxsize=1)
def load_data(path: str) -> pd.DataFrame:
    """Loads the clustered and categorized Amazon reviews DataFrame."""
    try:
        df = pd.read_csv(path)
        # Ensure necessary columns are correctly typed and handle missing values
        if 'asin' in df.columns:
            df['asin'] = df['asin'].astype(str)
        if 'full_review' in df.columns:
            df['full_review'] = df['full_review'].astype(str).fillna('')
        if 'meta_category' in df.columns:
            df['meta_category'] = df['meta_category'].astype(str).fillna('Uncategorized')
        if 'sentiment' in df.columns: # Assuming this column exists from sentiment analysis
            df['sentiment'] = df['sentiment'].astype(str)
        print(f"DataFrame loaded from {path}. Shape: {df.shape}")
        return df
    except FileNotFoundError:
        print(f"Error: Data file not found at {path}. Please check the path and ensure the file exists.")
        return pd.DataFrame()
    except Exception as e:
        print(f"Error loading data: {e}")
        return pd.DataFrame()

@lru_cache(maxsize=1)
def load_summarization_model(model_name: str):
    """Loads the T5 summarization model and tokenizer."""
    print(f"Loading summarization model: {model_name}...")
    try:
        tokenizer = T5TokenizerFast.from_pretrained(model_name)
        model = T5ForConditionalGeneration.from_pretrained(model_name).to(DEVICE)
        print("Summarization model and tokenizer loaded successfully.")
        return tokenizer, model
    except Exception as e:
        print(f"Error loading summarization model: {e}")
        print("Please ensure you have an internet connection and the model name is correct.")
        return None, None

df_clustered_categorized = load_data(CLUSTERED_DATA_PATH)
summarizer_tokenizer, summarizer_model = load_summarization_model(SUMMARIZATION_MODEL_NAME)

if df_clustered_categorized.empty or summarizer_tokenizer is None or summarizer_model is None:
    print("Exiting due to critical loading errors. Please resolve the issues above.")
    exit() # Exit the script if essential components are not loaded

# ==============================================================================
# 3) PRODUCT RECOMMENDATION LOGIC
# ==============================================================================
def get_top_products_by_category(df: pd.DataFrame, num_products_per_category: int = 3, min_reviews_threshold: int = 3): # Lowered threshold to 3
    """
    Identifies top products within each meta_category based on the proportion of positive reviews.
    Assumes 'full_review' is used for product identification as 'asin' is not available and 'sentiment' (positive/negative).
    """
    print(f"Identifying top {num_products_per_category} products per category with at least {min_reviews_threshold} reviews...")
    recommendations = {}

    for category in df['meta_category'].unique():
        category_df = df[df['meta_category'] == category].copy()

        if category_df.empty:
            continue

        # Calculate positive review count and total review count per product, grouping by 'full_review'
        product_sentiment = category_df.groupby('full_review').agg(
            positive_reviews=('sentiment', lambda x: (x == 'positive').sum()),
            total_reviews=('sentiment', 'count')
        ).reset_index()

        # Calculate the proportion of positive reviews
        product_sentiment['positive_ratio'] = product_sentiment['positive_reviews'] / product_sentiment['total_reviews']

        # Filter products with at least a minimum number of reviews to ensure significance
        reliable_products = product_sentiment[product_sentiment['total_reviews'] >= min_reviews_threshold]

        # Sort by positive ratio (and total reviews as a tie-breaker) and get top products
        top_products = reliable_products.sort_values(
            by=['positive_ratio', 'total_reviews'],
            ascending=[False, False]
        ).head(num_products_per_category)

        if not top_products.empty:
            # Store 'full_review' as product identifier since 'asin' is not available
            recommendations[category] = top_products.rename(columns={'full_review': 'product_identifier'}).to_dict(orient='records')
            print(f"  Top products for '{category}': {[p['product_identifier'][:50] + '...' for p in recommendations[category]]}") # Print snippet of review
        else:
            print(f"  No reliable top products found for category: '{category}'")

    return recommendations

product_recommendations_by_category = get_top_products_by_category(df_clustered_categorized)
print("\nGenerated Product Recommendations Summary:")
for cat, products in product_recommendations_by_category.items():
    print(f"- {cat}: {[p['product_identifier'][:50] + '...' for p in products]}") # Print snippet of review


# ==============================================================================
# 4) SUMMARIZATION FUNCTION (ADAPTED FROM main_text_summarization.ipynb)
# ==============================================================================
def summarize_text(text: str, tokenizer, model, max_length: int = 60, min_length: int = 20) -> str:
    """
    Generates a summary for the given text using the loaded T5 model.
    """
    if not isinstance(text, str) or not text.strip():
        return "No summary available."
    try:
        inputs = tokenizer(
            "summarize: " + text,
            return_tensors="pt",
            max_length=512, # Max input length for T5
            truncation=True
        ).to(DEVICE)

        summary_ids = model.generate(
            inputs.input_ids,
            max_length=max_length,
            min_length=min_length,
            length_penalty=2.0,
            num_beams=4,
            early_stopping=True
        )
        summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
        return summary
    except Exception as e:
        print(f"Error during summarization: {e}")
        return "Failed to generate summary."

# ==============================================================================
# 5) BLOG POST GENERATION
# ==============================================================================
def generate_blog_post(category: str, recommended_products: list, df_data: pd.DataFrame, tokenizer, model) -> str:
    """Generates a blog post for a given category and its recommended products."""
    post_title = f"🌟 Top Picks in {category}: Reviews You Can Trust!"
    post_content = f"# {post_title}\n\n"
    post_content += f"Looking for the best products in the **{category}** category? We've analyzed thousands of Amazon reviews, leveraged advanced sentiment analysis, and smart clustering to bring you the top-rated items that customers absolutely love. Our deep dive into customer feedback revealed these gems based on overwhelmingly positive experiences!\n\n"
    post_content += "--- \n\n"

    for product_info in recommended_products:
        # Use the product_identifier from the recommendation dict (which is 'full_review' in this case)
        product_identifier = product_info['product_identifier']
        # Use a snippet of the review text as a placeholder for product title
        product_title = f"Product based on review: \"{product_identifier[:70]}...\""


        # Get relevant positive reviews for summarization - filter by the 'full_review' text used as identifier
        product_positive_reviews = df_data[(df_data['full_review'] == product_identifier) & (df_data['sentiment'] == 'positive')]['full_review'].tolist()

        post_content += f"## ✨ {product_title}\n\n"
        post_content += f"**Representative Review:** `{product_identifier[:100]}...`\n\n" # Display a longer snippet of the review

        if product_positive_reviews:
            # Select up to 3 random positive reviews to summarize (will likely be the same review if grouping by full_review)
            sample_reviews = random.sample(product_positive_reviews, min(3, len(product_positive_reviews)))
            post_content += "**What customers are saying (summarized highlights):**\n\n"
            for review_text in sample_reviews:
                summary = summarize_text(review_text, tokenizer, model, max_length=50, min_length=15)
                post_content += f"> \"*{summary}*\"\n\n"
        else:
            post_content += "*(No prominent positive reviews found to highlight with summaries.)*\n\n"

        # Add a placeholder link (cannot create a real link without ASIN)
        post_content += "[Link to Product (ASIN not available)]\n\n"
        post_content += "--- \n\n"

    post_content += "We hope these data-driven recommendations empower you to make excellent purchasing decisions. Happy shopping, and may your next Amazon buy be a fantastic one!\n"
    return post_content

print("\nGenerating blog posts...")
generated_blog_posts = {}
for category, products_list in product_recommendations_by_category.items():
    if products_list: # Only generate if there are recommendations
        print(f"  Generating post for category: '{category}'")
        blog_post_text = generate_blog_post(category, products_list, df_clustered_categorized, summarizer_tokenizer, summarizer_model)
        generated_blog_posts[category] = blog_post_text
    else:
        print(f"  Skipping post generation for empty category: '{category}'")

# ==============================================================================
# 6) SAVE FINAL OUTPUT
# ==============================================================================
print("\nSaving generated blog posts...")
os.makedirs(OUTPUT_BLOG_POSTS_DIR, exist_ok=True)

for category, post_content in generated_blog_posts.items():
    # Sanitize category name for filename
    safe_category_name = category.replace(' ', '_').replace('/', '_').replace('&', 'and').lower()
    file_name = f"{OUTPUT_BLOG_POSTS_DIR}amazon_recommendations_{safe_category_name}.md"
    with open(file_name, 'w', encoding='utf-8') as f:
        f.write(post_content)
    print(f"  Saved blog post for '{category}' to: {file_name}")

print("\nAll blog posts generated and saved successfully!")
print(f"You can find them in your Google Drive at: {OUTPUT_BLOG_POSTS_DIR}")

Using device: cpu
Mounting Google Drive...
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Google Drive mounted successfully.
DataFrame loaded from /content/drive/MyDrive/amazon_reviews_clustered_and_categorized.csv. Shape: (47295, 5)
Loading summarization model: t5-small...
Summarization model and tokenizer loaded successfully.
Identifying top 3 products per category with at least 3 reviews...
  Top products for 'Miscellaneous Electronics': ['Christmas Gift bought fianc cant get enough awesom...', 'Good reader original kindle decided upgrade functi...', 'Good, but one small thing annoys me... purchased u...']
  Top products for 'E-readers & Tablets': ['Great for entertainment & e-mail awesome tablet fa...', 'Great tablet bought tablet daughter loves...', 'Great tablet bought tablet fiance daughter getting...']
  Top products for 'Smart Home Devices': ['Great, Cost-Effective Echo Device bought amazon ta...